In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../modules')
from seeDistinctValues import seeDistinctValues
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, precision_recall_fscore_support, confusion_matrix, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.dummy import DummyClassifier
from collections import Counter

df = pd.read_csv('merged_checked.csv')


# Definire l'header
header = [
    "Agreement Manuel vs. Angelo",
    "Agreement Manuel vs. Marinella",
    "Agreement Marinella vs. Angelo",
    "Agreement Manuel vs. ChatGPT",
    "Agreement Marinella vs. chatGPT",
    "Agreement Angelo vs. chatGPT",
    "Interpretazione K"
]

# Creare un DataFrame vuoto con l'header specificato
results = pd.DataFrame(columns=header)

In [2]:
df

,id,title,annotator,Q1.1,Q1.2,Q1.3,Q1.3.1,Q1.4,Q1.5,Q1.6,Q1.7,Q1.7.1
0,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Manuel,2;1;1;1;1,1,3,-1,2,4,4,1,-1
1,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Angelo,2;1;1;1;1,4,1;3,1,1,4,4,1,-1
2,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Manuel,2;1;1;1;1,4,1;3,-1,2,4,4,1,-1
3,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Marinella,2;1;1;1;1,4,1;3,-1,1,4,4,2,-1
4,34b92f4c-32db-454c-a369-ca2032443f6c,Le borsette fatte a 3 euro l'ora vendute a 820...,Manuel,1;2;1;1;1,4,3,-1,1,3,2,2,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
1014,3696bc79-2f24-47e2-9f13-fabd92b7a271,"Variante Delta, quanto sono efficaci i vaccini...",ChatGPT4AsAnnotator,2;1;1;1;1,4,1;3,2,1,4,4,1,6
1015,89ca8966-8ade-49fd-a036-c528a1f7871c,Tutti i passi falsi di Davigo: dal caso Palama...,ChatGPT4AsAnnotator,2;1;2;1;1,4,1;3;5,-1,1,4,4,1,6
1016,a425710b-96e3-4840-87be-75deea62775d,Quanto si preoccupa l'Ue per la Macron fatigue,ChatGPT4AsAnnotator,1;1;2;1;1,4,1;2,-1,1,4,4,1,6
1017,df86ebbe-1672-4c99-8895-b0fe742ed00c,CSM ED I VERBALI DI AMARA: I SEGRETI DELLA NUO...,ChatGPT4AsAnnotator,2;1;2;1;1,4,1;5;9,-1,1,3,3,1,6


# Funzioni

## Funzione classica per il calcolo del Kappa


In [3]:
def inter_annotator_agreement(df, column_name, annotator1, annotator2, optional, numerical = False):
    r"""
    The inter_annotator_agreement function calculates the inter-annotator agreement between 
    two annotators for a given column in a DataFrame using Cohen's kappa score.
    Parameters:
        df: The DataFrame containing the annotation data.
        column_name: The name of the column for which the agreement is being calculated.
        annotator1: The name of the first annotator.
        annotator2: The name of the second annotator.
        optional: A boolean indicating whether missing values (or specific placeholder values) should be excluded from the analysis.
        numerical: A boolean indicating whether the column contains numerical values.

    """
    # Filter rows for the given annotators
    df_annotator1 = df[df['annotator'] == annotator1]
    df_annotator2 = df[df['annotator'] == annotator2]
    
    # Merge dataframes on the ID column
    merged_df = df_annotator1.merge(df_annotator2, on='id')
    
    # Filter rows with non-missing values for the given column
    merged_df = merged_df.dropna(subset=[column_name+'_x', column_name+'_y'])

    # Rimuovo le righe vuote dal dataset degli annotatori
    if (not optional):
        merged_df = merged_df[merged_df[column_name+'_x'] != -1]
        merged_df = merged_df[merged_df[column_name+'_x'] != '-1']
        merged_df = merged_df[merged_df[column_name+'_y'] != -1]
        merged_df = merged_df[merged_df[column_name+'_y'] != '-1']
    
    if (numerical):
        agreement_score = cohen_kappa_score(merged_df[column_name+'_x'], merged_df[column_name+'_y'])
    else:
        # Converto le colonne in stringhe
        merged_df[column_name + '_x'] = merged_df[column_name + '_x'].astype(str)
        merged_df[column_name + '_y'] = merged_df[column_name + '_y'].astype(str)
        agreement_score = cohen_kappa_score(merged_df[column_name+'_x'], merged_df[column_name+'_y'])

    return(agreement_score)


## Funzione per le metriche standard (balanced accuracy)

In [4]:
def calculate_standard_metrics(annotator_1, annotator_2, df, optional=False):
    filtered_df = df[df['annotator'].isin([annotator_1, annotator_2])]
    ids_annotator_1 = filtered_df[filtered_df['annotator'] == annotator_1]['id']
    ids_annotator_2 = filtered_df[filtered_df['annotator'] == annotator_2]['id']
    common_ids = ids_annotator_1[ids_annotator_1.isin(ids_annotator_2)]
    df = filtered_df[filtered_df['id'].isin(common_ids)]

    gpt_df = df[df['annotator'] == annotator_2]
    human_df = df[df['annotator'] == annotator_1]

    results = {}

    merged_data = pd.merge(gpt_df, human_df, on="id", suffixes=("_gpt", "_human"))

    columns_of_interest = df.drop(columns=['id', 'annotator']).columns.tolist()

    plt.subplots_adjust(top=10)

    for col in columns_of_interest:
        y_gpt = merged_data[col + "_gpt"]
        y_human = merged_data[col + "_human"]

        if not optional:
            # Filter out cases where either annotator has value -1
            valid_indices = (y_gpt != -1) & (y_gpt != '-1') & (y_human != -1) & (y_human != '-1')
        else:
            valid_indices = pd.Series([True] * len(y_gpt))
        
        y_gpt = y_gpt[valid_indices]
        y_human = y_human[valid_indices]

        if len(y_gpt) == 0:
            continue  # Skip this column if there are no valid annotations
        
        # Baseline "most_frequent" (as you already have)
        baseline_model_most_frequent = DummyClassifier(strategy="most_frequent")
        # Filtering human_df based on the IDs present in merged_data with valid indices
        baseline_model_most_frequent.fit(human_df.loc[human_df['id'].isin(merged_data.loc[valid_indices, 'id']), [col]], y_human)
        y_pred_baseline_most_frequent = baseline_model_most_frequent.predict(human_df.loc[human_df['id'].isin(merged_data.loc[valid_indices, 'id']), [col]])

        # Baseline "stratified"
        baseline_model_stratified = DummyClassifier(strategy="stratified")
        # Same filtering for stratified baseline model
        baseline_model_stratified.fit(human_df.loc[human_df['id'].isin(merged_data.loc[valid_indices, 'id']), [col]], y_human)
        y_pred_baseline_stratified = baseline_model_stratified.predict(human_df.loc[human_df['id'].isin(merged_data.loc[valid_indices, 'id']), [col]])

        balanced_acc = balanced_accuracy_score(y_human, y_gpt)
        f1_micro = f1_score(y_human, y_gpt, average='micro')
        f1_macro = f1_score(y_human, y_gpt, average='macro')
        f1_avg = f1_score(y_human, y_gpt, average='weighted')

        balanced_acc_baseline_most_frequent = balanced_accuracy_score(y_human, y_pred_baseline_most_frequent)
        f1_micro_baseline_most_frequent = f1_score(y_human, y_pred_baseline_most_frequent, average='micro')
        f1_macro_baseline_most_frequent = f1_score(y_human, y_pred_baseline_most_frequent, average='macro')
        f1_avg_baseline_most_frequent = f1_score(y_human, y_pred_baseline_most_frequent, average='weighted')

        balanced_acc_baseline_stratified = balanced_accuracy_score(y_human, y_pred_baseline_stratified)
        f1_micro_baseline_stratified = f1_score(y_human, y_pred_baseline_stratified, average='micro')
        f1_macro_baseline_stratified = f1_score(y_human, y_pred_baseline_stratified, average='macro')
        f1_avg_baseline_stratified = f1_score(y_human, y_pred_baseline_stratified, average='weighted')

        results[col] = {
            'balanced_accuracy': balanced_acc,
            'f1_micro': f1_micro,
            'f1_macro': f1_macro,
            'f1_avg': f1_avg,
            'baseline_most_frequent_balanced_accuracy': balanced_acc_baseline_most_frequent,
            'baseline_most_frequent_f1_micro': f1_micro_baseline_most_frequent,
            'baseline_most_frequent_f1_macro': f1_macro_baseline_most_frequent,
            'baseline_most_frequent_f1_avg': f1_avg_baseline_most_frequent,
            'baseline_stratified_balanced_accuracy': balanced_acc_baseline_stratified,
            'baseline_stratified_f1_micro': f1_micro_baseline_stratified,
            'baseline_stratified_f1_macro': f1_macro_baseline_stratified,
            'baseline_stratified_f1_avg': f1_avg_baseline_stratified
        }
    num_cols = len(columns_of_interest)
    num_plots_per_row = 3
    num_rows = (num_cols + num_plots_per_row - 1) // num_plots_per_row

    fig, axs = plt.subplots(num_rows, num_plots_per_row, figsize=(18, 4*num_rows))

    for i, col in enumerate(columns_of_interest):
        metrics = results[col]

        labels = ['Bal. Accuracy', 'F1 Micro', 'F1 Macro', 'F1 Weighted']
        model_values = [metrics['balanced_accuracy'], metrics['f1_micro'], metrics['f1_macro'], metrics['f1_avg']]
        baseline_values_most_frequent = [metrics['baseline_most_frequent_balanced_accuracy'], metrics['baseline_most_frequent_f1_micro'],
                                         metrics['baseline_most_frequent_f1_macro'], metrics['baseline_most_frequent_f1_avg']]
        baseline_values_stratified = [metrics['baseline_stratified_balanced_accuracy'], metrics['baseline_stratified_f1_micro'],
                                      metrics['baseline_stratified_f1_macro'], metrics['baseline_stratified_f1_avg']]

        row_idx = i // num_plots_per_row
        col_idx = i % num_plots_per_row

        x = np.arange(len(labels))
        width = 0.25

        axs[row_idx, col_idx].bar(x - width, model_values, width, label='Model')
        axs[row_idx, col_idx].bar(x, baseline_values_most_frequent, width, label='Baseline (most frequent)')
        axs[row_idx, col_idx].bar(x + width, baseline_values_stratified, width, label='Baseline (stratified)')

        for j, (v, b1, b2) in enumerate(zip(model_values, baseline_values_most_frequent, baseline_values_stratified)):
            offset = 0
            axs[row_idx, col_idx].text(x[j] - width + offset, v + 0.01, f"{v:.4f}", ha='center', fontsize=7)
            axs[row_idx, col_idx].text(x[j], b1 + 0.01, f"{b1:.4f}", ha='center', fontsize=7)
            axs[row_idx, col_idx].text(x[j] + width + offset, b2 + 0.01, f"{b2:.4f}", ha='center', fontsize=7)

        axs[row_idx, col_idx].set_ylabel('Score')
        axs[row_idx, col_idx].set_title(f'Metrics for {col}')
        axs[row_idx, col_idx].set_xticks(x)
        axs[row_idx, col_idx].set_xticklabels(labels)
        axs[row_idx, col_idx].legend()
        axs[row_idx, col_idx].set_ylim(0, 1.07)

    for i in range(num_rows * num_plots_per_row):
        if i >= num_cols:
            axs.flatten()[i].axis('off')

    plt.tight_layout()
    plt.show()

## Confusion Matrix

In [5]:
def calculate_confusion_matrix(annotator_1, annotator_2, columns, possible_values):
    confusion_matrices = {}
    for column in columns:
        y_true = []
        y_pred = []

        articles = set(df[df['annotator'] == annotator_1]['id']).intersection(
            set(df[df['annotator'] == annotator_2]['id']))

        for article_id in articles:
            annotator1 = df[(df['id'] == article_id) & (df['annotator'] == annotator_1)][[column]]
            annotator2 = df[(df['id'] == article_id) & (df['annotator'] == annotator_2)][[column]]

            annotator1 = annotator1.astype(str)
            annotator2 = annotator2.astype(str)

            for col in annotator1.columns:
                annotations_1 = annotator1[col].values.flatten()
                annotations_2 = annotator2[col].values.flatten()
                for ann_1, ann_2 in zip(annotations_1, annotations_2):
                    y_true.append(ann_1)
                    y_pred.append(ann_2)

        confusion_matrices[column] = confusion_matrix(y_true, y_pred, labels=possible_values[column])
    plot_confusion_matrices(confusion_matrices, possible_values)
    return 0

def plot_confusion_matrices(conf_matrices, possible_values):
    num_cols = len(conf_matrices)
    num_rows = int(np.ceil(num_cols / 4))
    fig, axes = plt.subplots(num_rows, 4, figsize=(20, num_rows * 4))

    # If there's only one row, axes may not be a 2D array
    if num_rows == 1:
        axes = np.expand_dims(axes, axis=0)
    
    for i, (column, conf_matrix) in enumerate(conf_matrices.items()):
        row = i // 4
        col = i % 4
        sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d", cbar=False, ax=axes[row, col], 
                    xticklabels=possible_values[column], yticklabels=possible_values[column])
        axes[row, col].set_xlabel("Predicted labels")
        axes[row, col].set_ylabel("True labels")
        axes[row, col].set_title(f"Confusion Matrix for Column '{column}'")

    # Hide any unused subplots
    for j in range(i + 1, num_rows * 4):
        fig.delaxes(axes.flatten()[j])

    plt.tight_layout()
    plt.show()

columns_to_consider = ["Q1.2", "Q1.3.1", "Q1.4", "Q1.5", "Q1.6", "Q1.7"]
possible_values = {
    'Q1.2': ['1', '2', '3', '4'],
    'Q1.3.1': ['1', '2'],
    'Q1.4': ['1', '2'],
    'Q1.5': ['1', '2', '3', '4'],
    'Q1.6': ['1', '2', '3', '4'],
    'Q1.7': ['1', '2'],
}


# Q 1.1 

## Questions

### Original

Q1.1: What type of news article are you reviewing? (Check one): <br>
    1.  Straight news, or 'hard news' <br>
    2.  Editorial/opinion <br>
    3.  Feature/investigation <br>
    4.  Satire <br>
    5.  Lifestyle content, obituary, 'listicle', or other 'soft news' <br>


### Revisited

Step 2 -  Is this article Straight news, or 'hard news'?(if the article reports essential information in a concise and impartial manner,  the information should be based on facts, and there should be no personal opinion of the author)<br>
  1. Yes<br>
  2. No<br>

<br>
Step 3 - Is this article an Editorial/opinion?(if the article communicates the author’s personal point of view about current events and topics in order to develop an argument about an issue and potentially sway readers’ opinions.​​ Opinion pieces include editorials, op-eds, commentary, analysis, and letters to the editor.)<br>
  1. Yes<br>
  2. No<br>


Step 4 - Is this article a Feature/investigation? (if the article contains at least one of the following elements: in-depth coverage, rich storytelling, and investigative elements)<br>
  1. Yes<br>
  2. No<br>
<br>

Step 5 - Is this article a  Satire?<br>
  1. Yes<br>
  2. No<br>


Step 6 - Is this article Lifestyle content, obituary, 'listicle', or other 'soft news' or gossip?<br>
  1. Yes<br>
  2. No<br>

## Analisi

### K

In [6]:
opzionale = False
print(inter_annotator_agreement(df, 'Q1.1', 'Manuel', 'Angelo', opzionale))
print(inter_annotator_agreement(df, 'Q1.1', 'Manuel', 'Marinella', opzionale))
print(inter_annotator_agreement(df, 'Q1.1', 'Angelo', 'Marinella', opzionale))
print('----------------------------------')
print(inter_annotator_agreement(df, 'Q1.1', 'Manuel', 'ChatGPT4AsAnnotator', opzionale))
print(inter_annotator_agreement(df, 'Q1.1', 'Marinella', 'ChatGPT4AsAnnotator', opzionale))
print(inter_annotator_agreement(df, 'Q1.1', 'Angelo', 'ChatGPT4AsAnnotator', opzionale))



0.6903472443453329
0.7307060755336618
0.8258518117901568
----------------------------------
0.43999094934867633
0.3768859271461197
0.36874032008260194


In [7]:
results

,Agreement Manuel vs. Angelo,Agreement Manuel vs. Marinella,Agreement Marinella vs. Angelo,Agreement Manuel vs. ChatGPT,Agreement Marinella vs. chatGPT,Agreement Angelo vs. chatGPT,Interpretazione K


### Vedo dove e come sbaglia

In [8]:

def custom_agreement(column_name,set1, set2):

    # Split the numbers
    set1 = set1.split(";")
    set2 = set2.split(";")

    # Verifica se entrambe le liste contengono almeno un 2
    if '2' in set1 and '2' in set2:
        # Verifica se i 2 sono nella stessa posizione
        if set1 == set2:
            return "Perfect Agreement (All correct)"
        # Verifica se nella seconda lista c'è almeno un 2 nella stessa posizione
        elif any(a == b == '2' for a, b in zip(set1, set2)):
            #print ('lista 1' +str(set1) + ' lista 2' + str(set2))
            return "Partial Agreement (Extra Incorrect)"
        else:
            return "No Agreement (All Incorrect)"
    else:
        return "No Agreement (All Incorrect)"


def discoverValues(df, column_name, annotator1, annotator2):
    # Creo l'oggetto per contare le divergenze
    divergences = Counter()

    df_annotator1 = df[df['annotator'] == annotator1]
    df_annotator2 = df[df['annotator'] == annotator2]
    
    merged_df = df_annotator1.merge(df_annotator2, on='id')
    merged_df = merged_df.dropna(subset=[column_name+'_x', column_name+'_y'])
    
    perfect_agreement_count = 0
    partial_correct_count = 0

    partial_extra_count = 0
    no_agreement_count = 0
    
    for index, row in merged_df.iterrows():
        if (row[column_name+'_x'] != -1 and row[column_name+'_x'] != '-1'):
            agreement_category = custom_agreement(column_name,row[column_name+'_x'], row[column_name+'_y'])
            if agreement_category == 'Perfect Agreement (All correct)':
                perfect_agreement_count += 1
            elif agreement_category == 'Partial Agreement (Some Correct)':
                partial_correct_count += 1
            elif agreement_category == 'Partial Agreement (Extra Incorrect)':
                partial_extra_count += 1
            elif agreement_category == 'No Agreement (All Incorrect)':
                no_agreement_count += 1

    print(annotator1 + ' e ' + annotator2 + ' per ' + column_name + ':')

    print('   Perfect Agreement:', perfect_agreement_count)
    if (column_name != 'Q1.1'):
        print('   Partial Agreement (Some Correct):', partial_correct_count)
    print('   Partial Agreement (Extra Incorrect):', partial_extra_count)
    print('   No Agreement (All Incorrect):', no_agreement_count)
    print('\n')


discoverValues(df, 'Q1.1', 'Manuel', 'ChatGPT4AsAnnotator')

Manuel e ChatGPT4AsAnnotator per Q1.1:
   Perfect Agreement: 156
   Partial Agreement (Extra Incorrect): 28
   No Agreement (All Incorrect): 47




In [9]:
df['Q1.1'].unique

<bound method Series.unique of 0       2;1;1;1;1
1       2;1;1;1;1
2       2;1;1;1;1
3       2;1;1;1;1
4       1;2;1;1;1
          ...    
1014    2;1;1;1;1
1015    2;1;2;1;1
1016    1;1;2;1;1
1017    2;1;2;1;1
1018    1;2;1;1;1
Name: Q1.1, Length: 1019, dtype: object>

### Cerco pattern di errori comuni

In [10]:
def discoverValues(df, column_name, annotator1, annotator2):
    # Creo l'oggetto per contare le divergenze
    divergences = Counter()

    df_annotator1 = df[df['annotator'] == annotator1]
    df_annotator2 = df[df['annotator'] == annotator2]
    
    merged_df = df_annotator1.merge(df_annotator2, on='id')
    merged_df = merged_df.dropna(subset=[column_name+'_x', column_name+'_y'])
    
    for index, row in merged_df.iterrows():
        if (row[column_name+'_x'] != -1 and row[column_name+'_x'] != '-1'):
            agreement_category = custom_agreement(column_name, row[column_name+'_x'], row[column_name+'_y'])
            if agreement_category == 'Partial Agreement (Extra Incorrect)':
                divergences[(tuple(row[column_name+'_x'].split(';')), tuple(row[column_name+'_y'].split(';')))] += 1

    # Ritorna i primi 10 casi più frequenti di divergenza
    return divergences.most_common(10)

# Modifica il chiamato di discoverValues per salvarne i risultati
divergences = discoverValues(df, 'Q1.1', 'Manuel', 'ChatGPT4AsAnnotator')
print("Casi più frequenti di divergenza tra la prima e la seconda lista:")
for divergence, count in divergences:
    print("Divergenza:", divergence, " - Frequenza:", count)

Casi più frequenti di divergenza tra la prima e la seconda lista:
Divergenza: (('2', '1', '1', '1', '1'), ('2', '1', '2', '1', '1'))  - Frequenza: 10
Divergenza: (('1', '1', '2', '1', '1'), ('2', '1', '2', '1', '1'))  - Frequenza: 9
Divergenza: (('1', '1', '2', '1', '2'), ('1', '1', '1', '1', '2'))  - Frequenza: 7
Divergenza: (('1', '1', '2', '1', '2'), ('1', '1', '2', '1', '1'))  - Frequenza: 1
Divergenza: (('1', '1', '2', '1', '1'), ('1', '1', '2', '1', '2'))  - Frequenza: 1


   Perfect Agreement: 105 + 41 <br>
   Partial Agreement (Extra Incorrect): 0<br>
   No Agreement (All Incorrect): 50 + 25<br>

### Considero valide le righe in cui Chat gpt ha messo (massimo e minimo) due valori e in cui uno dei due valori è corrretto

In [11]:
new_column_name = "Q1.1_Fixed"
df[new_column_name] = df["Q1.1"]


for index, row in df.iterrows():
    annotator = row["annotator"]
    q1_1 = row["Q1.1"]
    id = row["id"]

# Divergenza: (('2', '1', '1', '1', '1'), ('2', '1', '2', '1', '1'))  - Frequenza: 41

    if annotator == "Manuel" and q1_1 == "2;1;1;1;1":
        for index2, row2 in df.iterrows():
            if row2["id"] == id and row2["annotator"] == "ChatGPT4AsAnnotator" and row2["Q1.1"] == "2;1;2;1;1":
                df.loc[index2, new_column_name] = "2;1;1;1;1"

# Divergenza: (('1', '1', '2', '1', '1'), ('2', '1', '2', '1', '1'))  - Frequenza: 25

    if annotator == "Manuel" and q1_1 == "1;1;2;1;1":
        for index2, row2 in df.iterrows():
            if row2["id"] == id and row2["annotator"] == "ChatGPT4AsAnnotator" and row2["Q1.1"] == "2;1;1;1;2":
                df.loc[index2, new_column_name] = "1;1;2;1;1"

# Divergenza: (('2', '1', '1', '1', '1'), ('2', '1', '1', '1', '2'))  - Frequenza: 5

    if annotator == "Manuel" and q1_1 == "2;1;1;1;1":
        for index2, row2 in df.iterrows():
            if row2["id"] == id and row2["annotator"] == "ChatGPT4AsAnnotator" and row2["Q1.1"] == "2;1;1;1;2":
                df.loc[index2, new_column_name] = "2;1;1;1;1"

# Divergenza: (('1', '2', '1', '1', '1'), ('2', '2', '1', '1', '1'))  - Frequenza: 2

    if annotator == "Manuel" and q1_1 == "1;2;1;1;1":
        for index2, row2 in df.iterrows():
            if row2["id"] == id and row2["annotator"] == "ChatGPT4AsAnnotator" and row2["Q1.1"] == "2;2;1;1;1":
                df.loc[index2, new_column_name] = "1;2;1;1;1"

# Divergenza: (('1', '1', '2', '1', '1'), ('1', '2', '2', '1', '1'))  - Frequenza: 1

    if annotator == "Manuel" and q1_1 == "1;1;2;1;1":
        for index2, row2 in df.iterrows():
            if row2["id"] == id and row2["annotator"] == "ChatGPT4AsAnnotator" and row2["Q1.1"] == "1;2;2;1;1":
                df.loc[index2, new_column_name] = "1;1;2;1;1"

# Divergenza: (('2', '1', '1', '1', '1'), ('2', '2', '1', '1', '1'))  - Frequenza: 1

    if annotator == "Manuel" and q1_1 == "2;1;1;1;1":
        for index2, row2 in df.iterrows():
            if row2["id"] == id and row2["annotator"] == "ChatGPT4AsAnnotator" and row2["Q1.1"] == "2;2;1;1;1":
                df.loc[index2, new_column_name] = "2;1;1;1;1"


df['Q1.1_Fixed']

0       2;1;1;1;1
1       2;1;1;1;1
2       2;1;1;1;1
3       2;1;1;1;1
4       1;2;1;1;1
          ...    
1014    2;1;1;1;1
1015    2;1;2;1;1
1016    1;1;2;1;1
1017    2;1;2;1;1
1018    1;2;1;1;1
Name: Q1.1_Fixed, Length: 1019, dtype: object

### Faccio nuovamente il K

In [12]:
opzionale = False
results.loc[0] = [
    inter_annotator_agreement(df, 'Q1.1', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df, 'Q1.1', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.1', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.1_Fixed', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.1_Fixed', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.1_Fixed', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]

In [13]:
results

,Agreement Manuel vs. Angelo,Agreement Manuel vs. Marinella,Agreement Marinella vs. Angelo,Agreement Manuel vs. ChatGPT,Agreement Marinella vs. chatGPT,Agreement Angelo vs. chatGPT,Interpretazione K
0,0.690347,0.730706,0.825852,0.490447,0.392159,0.394659,


# Q 1.2

## Questions

### Original

Q1.2: How accurately does the story’s headline describe the content of the story? (Check one): <br>
    1.  Extremely inaccurately<br>
    2.  Somewhat inaccurately<br>
    3.  Somewhat accurately<br>
    4.  Extremely accurately<br>


### Refactored

Does the content of this news reflect what you expect from reading the headline?<br>
    1.  No<br>
    4.  Yes<br>

## Analisi

In [14]:
df['Q1.2']

0       1
1       4
2       4
3       4
4       4
       ..
1014    4
1015    4
1016    4
1017    4
1018    4
Name: Q1.2, Length: 1019, dtype: int64

In [15]:
df['Q1.2'].unique()

array([ 1,  4, -1])

### K

In [16]:
opzionale = False
numerica = False

results.loc[1] = [
    inter_annotator_agreement(df, 'Q1.2', 'Manuel', 'Angelo', opzionale, numerica),
    inter_annotator_agreement(df, 'Q1.2', 'Manuel', 'Marinella', opzionale, numerica),
    inter_annotator_agreement(df, 'Q1.2', 'Angelo', 'Marinella', opzionale, numerica),
    inter_annotator_agreement(df, 'Q1.2', 'Manuel', 'ChatGPT4AsAnnotator', opzionale, numerica),
    inter_annotator_agreement(df, 'Q1.2', 'Marinella', 'ChatGPT4AsAnnotator', opzionale, numerica),
    inter_annotator_agreement(df, 'Q1.2', 'Angelo', 'ChatGPT4AsAnnotator', opzionale, numerica),
    ""
]


In [17]:
results

,Agreement Manuel vs. Angelo,Agreement Manuel vs. Marinella,Agreement Marinella vs. Angelo,Agreement Manuel vs. ChatGPT,Agreement Marinella vs. chatGPT,Agreement Angelo vs. chatGPT,Interpretazione K
0,0.690347,0.730706,0.825852,0.490447,0.392159,0.394659,
1,0.452055,0.355599,0.428279,0.219982,0.233129,0.169449,


# Q 1.3

## Questions

### Original

Q1.3: Does the story’s headline contain any of the following elements? (Check all that apply):<br>
    1.  A proper noun<br>
    2.  A question<br>
    3.  A colon<br>
    4.  A quotation<br>
    5.  Words or phrases that are entirely capitalized (all caps)<br>
    6.  The pronoun 'this'<br>
    7.  Generalizing terms (ex: 'never', 'always')<br>
    8.  Explanation phrases, such as 'Here’s why...'<br>
    9.  Hyperbolic, emotional or sensationalized language<br>


### Revisited

Step 2 - Does the title contain a proper noun?<br>
 1. No<br>
 2. Yes<br>


Step 3 - Does the title contain a question?<br>
 1. No<br>
 2. Yes<br>


Step 4 - Does the title contain the colon character (:)?<br>
 1. No<br>
 2. Yes<br>


Step 5 - Does the title contain quoted text?<br>
 1. No<br>
 2. Yes<br>


Step 6 - Does the title contain at least one word written in capital letters?<br>
 1. No<br>
 2. Yes<br>


Step 7 - Does the title contain the pronoun 'questo' and its derivatives?<br>
 1. No<br>
 2. Yes<br>


Step 8 - Does the title contain generalizing terms (such as 'mai', 'sempre')?<br>
 1. No<br>
 2. Yes<br>


Step 9 - Does the title contain explanation phrases, such as 'Ecco perché...'?<br>
 1. No<br>
 2. Yes<br>


Step 10 - Does the title contain hyperbolic, emotional, or sensationalistic language?<br>
 1.  No<br>
 2.  Yes<br>


## Analisi

In [18]:
df['Q1.3'].unique()

array(['3', '1;3', '3;9', '1;3;5', '1;3;4', '1;4', '4;9', '3;4', '1', '9',
       '-1', '1;3;9', '1;9', '1;5', '1;2;4', '1;2;3;4', '1;3;8', '1;8',
       '1;3;4;9', '1;2;3;4;9', '1;3;4;7;9', '3;4;9', '1;2', '1;3;5;9',
       '3;4;8', '7', '2;3', '1;5;9', '4;8', '8', '3;8', '3;6', '4', '5',
       '2', '1;2;9', '1;4;8;9', '1;3;4;5;9', '1;2;3;4;5;9', '1;2;8;9',
       '1;3;8;9', '1;2;4;9', '1;4;9', '1;7;9', '1;8;9', '1;4;5;9',
       '1;3;4;5;6;9', '1;4;5', '1;4;7', '1;2;3;8', '1;2;3', '1;3;5;8;9',
       '1;2;3;5;9', '1;2;3;8;9', '1;3;4;5', '1;5;8'], dtype=object)

In [19]:
opzionale = True

print(inter_annotator_agreement(df, 'Q1.3', 'Manuel', 'Angelo', opzionale))
print(inter_annotator_agreement(df, 'Q1.3', 'Manuel', 'Marinella', opzionale))
print(inter_annotator_agreement(df, 'Q1.3', 'Angelo', 'Marinella', opzionale))
print('----------------------------------')
print(inter_annotator_agreement(df, 'Q1.3', 'Manuel', 'ChatGPT4AsAnnotator', opzionale))
print(inter_annotator_agreement(df, 'Q1.3', 'Marinella', 'ChatGPT4AsAnnotator', opzionale))
print(inter_annotator_agreement(df, 'Q1.3', 'Angelo', 'ChatGPT4AsAnnotator', opzionale))

0.591187032903038
0.7006864298106661
0.6189028420356907
----------------------------------
0.1975641910979975
0.22891791480711277
0.23535960810509904


### Creo colonne singole per ciascun valore possibile nella colonna e salvo in un nuovo dataframe

In [20]:
# Dividi la colonna 'Q1.3' in colonne separate
df_split = df['Q1.3'].str.get_dummies(sep=';')

# Rinomina le colonne con i valori unici
df_split.columns = ['Q1.3_' + col for col in df_split.columns]

# Unisci il dataframe diviso con il dataframe originale
df_1_3 = pd.concat([df, df_split], axis=1)

# Mostra il dataframe risultante
df_1_3 = df_1_3.loc[:, ['id','annotator','Q1.3_-1','Q1.3_1','Q1.3_2','Q1.3_3','Q1.3_4','Q1.3_5','Q1.3_6','Q1.3_7','Q1.3_8','Q1.3_9']]

df_1_3

,id,annotator,Q1.3_-1,Q1.3_1,Q1.3_2,Q1.3_3,Q1.3_4,Q1.3_5,Q1.3_6,Q1.3_7,Q1.3_8,Q1.3_9
0,18d71db7-f362-46fa-953e-d8353c7f7232,Manuel,0,0,0,1,0,0,0,0,0,0
1,18d71db7-f362-46fa-953e-d8353c7f7232,Angelo,0,1,0,1,0,0,0,0,0,0
2,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Manuel,0,1,0,1,0,0,0,0,0,0
3,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Marinella,0,1,0,1,0,0,0,0,0,0
4,34b92f4c-32db-454c-a369-ca2032443f6c,Manuel,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1014,3696bc79-2f24-47e2-9f13-fabd92b7a271,ChatGPT4AsAnnotator,0,1,0,1,0,0,0,0,0,0
1015,89ca8966-8ade-49fd-a036-c528a1f7871c,ChatGPT4AsAnnotator,0,1,0,1,0,1,0,0,0,0
1016,a425710b-96e3-4840-87be-75deea62775d,ChatGPT4AsAnnotator,0,1,1,0,0,0,0,0,0,0
1017,df86ebbe-1672-4c99-8895-b0fe742ed00c,ChatGPT4AsAnnotator,0,1,0,0,0,1,0,0,0,1


### Rifaccio il K

In [21]:
opzionale = True



results.loc[2] = [
    inter_annotator_agreement(df_1_3, 'Q1.3_1', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_1', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_1', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_1', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_1', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_1', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]
results.loc[3] = [
    inter_annotator_agreement(df_1_3, 'Q1.3_2', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_2', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_2', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_2', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_2', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_2', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]
results.loc[4] = [
    inter_annotator_agreement(df_1_3, 'Q1.3_3', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_3', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_3', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_3', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_3', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_3', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]

results.loc[5] = [
    inter_annotator_agreement(df_1_3, 'Q1.3_4', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_4', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_4', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_4', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_4', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_4', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]
results.loc[6] = [
    inter_annotator_agreement(df_1_3, 'Q1.3_5', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_5', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_5', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_5', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_5', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_5', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]
results.loc[7] = [
    inter_annotator_agreement(df_1_3, 'Q1.3_6', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_6', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_6', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_6', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_6', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_6', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]
results.loc[8] = [
    inter_annotator_agreement(df_1_3, 'Q1.3_7', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_7', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_7', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_7', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_7', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_7', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]
results.loc[9] = [
    inter_annotator_agreement(df_1_3, 'Q1.3_8', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_8', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_8', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_8', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_8', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_8', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]
results.loc[10] = [
    inter_annotator_agreement(df_1_3, 'Q1.3_9', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_9', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_9', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_9', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_9', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_3, 'Q1.3_9', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]


results

/home/john/Progetti/ChatGPTAsAnnotator/ChatGPTAsAnnotator/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/john/Progetti/ChatGPTAsAnnotator/ChatGPTAsAnnotator/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:708: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/john/Progetti/ChatGPTAsAnnotator/ChatGPTAsAnnotator/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/john/Progetti/ChatGPTAsAnnotator/ChatGPTAsAnnotator/.venv/lib/python3.12/site-packages/sklearn/metrics/_c

,Agreement Manuel vs. Angelo,Agreement Manuel vs. Marinella,Agreement Marinella vs. Angelo,Agreement Manuel vs. ChatGPT,Agreement Marinella vs. chatGPT,Agreement Angelo vs. chatGPT,Interpretazione K
0,0.690347,0.730706,0.825852,0.490447,0.392159,0.394659,
1,0.452055,0.355599,0.428279,0.219982,0.233129,0.169449,
2,0.735294,0.778922,0.761538,0.045519,0.039092,0.044853,
3,0.266304,0.853081,0.655556,0.484663,0.409792,0.418538,
4,0.870370,0.916532,0.889993,0.293578,0.364916,0.274823,
5,0.859130,0.873469,0.920851,0.706326,0.845737,0.757073,
6,0.492958,0.663043,0.661818,0.238566,0.172932,0.143342,
7,NaN,0.000000,NaN,NaN,-0.004566,0.000000,
8,0.000000,NaN,0.000000,0.000000,0.393382,-0.004975,
9,0.390977,0.884758,0.728070,0.491749,0.550409,0.550667,


### Faccio la balanced accuracy

In [22]:
optional = False
#calculate_standard_metrics('Marinella', 'ChatGPT4AsAnnotator', df_1_3, optional)

In [23]:
#calculate_standard_metrics('Angelo', 'ChatGPT4AsAnnotator', df_1_3, optional)

In [24]:
#calculate_standard_metrics('Manuel', 'ChatGPT4AsAnnotator', df_1_3, optional)

# Q 1.3.1

## Questions

### Original

Q1.3.1: If the headline contains a quotation, does that same quotation appear in the content of the story? (Check one):<br>
    1.  No<br>
    2.  Yes<br>
    -1. No quotation present<br>


### Reviseted

Step 2 - If the article’s title contains quoted text, does that same quoted text appear in the story's content? (Check one):<br>
    1.  No<br>
    2.  Yes<br>
    -1. No quotation present<br>


## Analisi

In [25]:
df['Q1.3.1'].unique()

array([-1,  1,  2])

In [26]:
opzionale = True



results.loc[11] = [
    inter_annotator_agreement(df, 'Q1.3.1', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df, 'Q1.3.1', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.3.1', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.3.1', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.3.1', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.3.1', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]


results


,Agreement Manuel vs. Angelo,Agreement Manuel vs. Marinella,Agreement Marinella vs. Angelo,Agreement Manuel vs. ChatGPT,Agreement Marinella vs. chatGPT,Agreement Angelo vs. chatGPT,Interpretazione K
0,0.690347,0.730706,0.825852,0.490447,0.392159,0.394659,
1,0.452055,0.355599,0.428279,0.219982,0.233129,0.169449,
2,0.735294,0.778922,0.761538,0.045519,0.039092,0.044853,
3,0.266304,0.853081,0.655556,0.484663,0.409792,0.418538,
4,0.870370,0.916532,0.889993,0.293578,0.364916,0.274823,
5,0.859130,0.873469,0.920851,0.706326,0.845737,0.757073,
6,0.492958,0.663043,0.661818,0.238566,0.172932,0.143342,
7,NaN,0.000000,NaN,NaN,-0.004566,0.000000,
8,0.000000,NaN,0.000000,0.000000,0.393382,-0.004975,
9,0.390977,0.884758,0.728070,0.491749,0.550409,0.550667,


### CM

In [27]:
columns_to_consider = ["Q1.3.1"]
possible_values = {
    'Q1.3.1': ['-1', '1.0', '2.0'],
}

#conf_matrices = calculate_confusion_matrix('Manuel','ChatGPT4AsAnnotator', columns_to_consider, possible_values)


# Q 1.4

## Questions

### Original

Q1.4: Does the article begin with a fact-based lead? (Check one):<br>
    1.  No<br>
    2.  Yes<br>


### Revisited

Step 11 - Does the first sentence of the text of this article clearly summarize the main fact that a news article is about by answering the where, when, how, and why of the fact described?<br><br>
No<br>
Yes<br>


## Analisi

In [28]:
df['Q1.4'].unique()

array([ 2,  1, -1])

### K

In [29]:


opzionale = True

results.loc[12] = [
    inter_annotator_agreement(df, 'Q1.4', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df, 'Q1.4', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.4', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.4', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.4', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.4', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]


results


,Agreement Manuel vs. Angelo,Agreement Manuel vs. Marinella,Agreement Marinella vs. Angelo,Agreement Manuel vs. ChatGPT,Agreement Marinella vs. chatGPT,Agreement Angelo vs. chatGPT,Interpretazione K
0,0.690347,0.730706,0.825852,0.490447,0.392159,0.394659,
1,0.452055,0.355599,0.428279,0.219982,0.233129,0.169449,
2,0.735294,0.778922,0.761538,0.045519,0.039092,0.044853,
3,0.266304,0.853081,0.655556,0.484663,0.409792,0.418538,
4,0.870370,0.916532,0.889993,0.293578,0.364916,0.274823,
5,0.859130,0.873469,0.920851,0.706326,0.845737,0.757073,
6,0.492958,0.663043,0.661818,0.238566,0.172932,0.143342,
7,NaN,0.000000,NaN,NaN,-0.004566,0.000000,
8,0.000000,NaN,0.000000,0.000000,0.393382,-0.004975,
9,0.390977,0.884758,0.728070,0.491749,0.550409,0.550667,


### CM

In [30]:
columns_to_consider = ["Q1.4"]
possible_values = {
    'Q1.4': ['-1', '1', '2'],
}

#conf_matrices = calculate_confusion_matrix('Manuel','ChatGPT4AsAnnotator', columns_to_consider, possible_values)


# Q 1.5

## Questions

### Original

Q1.5 Please rate the degree of bias in the article copy (Check one):<br>
    1.  Extremely biased<br>
    2.  Mostly biased<br>
    3.  Somewhat biased<br>
    4.  Entirely unbiased<br>


### Revisited

Step 12 - Please rate the degree of bias in the article copy:<br>
    1.  Extremely Biased<br>
    2.  Biased<br>
    3.  Partly biased<br>
    4.  Unbiased<br>


## Analisi

### K

In [31]:
opzionale = False
numerica = True

print(inter_annotator_agreement(df, 'Q1.5', 'Manuel', 'Angelo', opzionale, numerica))
print(inter_annotator_agreement(df, 'Q1.5', 'Manuel', 'Marinella', opzionale, numerica))
print(inter_annotator_agreement(df, 'Q1.5', 'Angelo', 'Marinella', opzionale, numerica))
print('----------------------------------')
print(inter_annotator_agreement(df, 'Q1.5', 'Manuel', 'ChatGPT4AsAnnotator', opzionale, numerica))
print(inter_annotator_agreement(df, 'Q1.5', 'Marinella', 'ChatGPT4AsAnnotator', opzionale, numerica))
print(inter_annotator_agreement(df, 'Q1.5', 'Angelo', 'ChatGPT4AsAnnotator', opzionale, numerica))


0.5023041474654377
0.4258468925046679
0.5871254162042175
----------------------------------
0.2926165362368153
0.39038970521128047
0.2797370327037718


### CM

In [32]:

columns_to_consider = ["Q1.5"]
possible_values = {
    'Q1.5': ['1', '2', '3', '4'],
}

#conf_matrices = calculate_confusion_matrix('Manuel','ChatGPT4AsAnnotator', columns_to_consider, possible_values)


### Trasformo in binaria

In [33]:
# Sostituisci i valori 3 con 4 e i valori 2 con 1 nella nuova colonna
df['Q1.5_binary'] = df['Q1.5'].replace({3: 4, 2: 1})

# Elimina i valori -1
df = df[df['Q1.5_binary'] != -1]

df['Q1.5_binary'].unique()


array([4, 1])

### Calcolo di nuovo le performance

In [34]:
opzionale = False


results.loc[13] = [
    inter_annotator_agreement(df, 'Q1.5_binary', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df, 'Q1.5_binary', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.5_binary', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.5_binary', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.5_binary', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.5_binary', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]


results



,Agreement Manuel vs. Angelo,Agreement Manuel vs. Marinella,Agreement Marinella vs. Angelo,Agreement Manuel vs. ChatGPT,Agreement Marinella vs. chatGPT,Agreement Angelo vs. chatGPT,Interpretazione K
0,0.690347,0.730706,0.825852,0.490447,0.392159,0.394659,
1,0.452055,0.355599,0.428279,0.219982,0.233129,0.169449,
2,0.735294,0.778922,0.761538,0.045519,0.039092,0.044853,
3,0.266304,0.853081,0.655556,0.484663,0.409792,0.418538,
4,0.870370,0.916532,0.889993,0.293578,0.364916,0.274823,
5,0.859130,0.873469,0.920851,0.706326,0.845737,0.757073,
6,0.492958,0.663043,0.661818,0.238566,0.172932,0.143342,
7,NaN,0.000000,NaN,NaN,-0.004566,0.000000,
8,0.000000,NaN,0.000000,0.000000,0.393382,-0.004975,
9,0.390977,0.884758,0.728070,0.491749,0.550409,0.550667,


# Q 1.6

## Questions

### Original

Q1.6 Does the article use sensationalised language? (Check one):<br>
    1.  Extremely sensationalised<br>
    2.  Somewhat sensationalised<br>
    3.  Mainly neutral<br>
    4.  Entirely neutral<br>


### Reviseted

Step 13 - Does the article use sensationalized language?:<br>
    1.  Extremely sensationalised<br>
    2.  Sensationalised<br>
    3.  Partly neutral<br>
    4.  Neutral<br>


## Analisi

In [35]:
df['Q1.6'].unique()

array([4, 2, 3, 1])

### K

In [36]:
opzionale = False
numerica = True

print(inter_annotator_agreement(df, 'Q1.6', 'Manuel', 'Angelo', opzionale, numerica))
print(inter_annotator_agreement(df, 'Q1.6', 'Manuel', 'Marinella', opzionale, numerica))
print(inter_annotator_agreement(df, 'Q1.6', 'Angelo', 'Marinella', opzionale, numerica))
print('----------------------------------')
print(inter_annotator_agreement(df, 'Q1.6', 'Manuel', 'ChatGPT4AsAnnotator', opzionale, numerica))
print(inter_annotator_agreement(df, 'Q1.6', 'Marinella', 'ChatGPT4AsAnnotator', opzionale, numerica))
print(inter_annotator_agreement(df, 'Q1.6', 'Angelo', 'ChatGPT4AsAnnotator', opzionale, numerica))


0.5731225296442688
0.4629577464788732
0.579470947320823
----------------------------------
0.3066409390577264
0.2563667232597623
0.3110752109136601



### CM

In [37]:
columns_to_consider = ["Q1.6"]
possible_values = {
    'Q1.6': ['-1', '1', '2', '3', '4'],
}

#conf_matrices = calculate_confusion_matrix('Manuel','ChatGPT4AsAnnotator', columns_to_consider, possible_values)


### Trasformo in binaria

In [38]:
# Sostituisci i valori 3 con 4 e i valori 2 con 1 nella nuova colonna
df['Q1.6_binary'] = df['Q1.6'].replace({3: 4, 2: 1})

# Elimina i valori -1
df = df[df['Q1.6_binary'] != -1]

df['Q1.6_binary'].unique()


/tmp/ipykernel_182339/1858030754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Q1.6_binary'] = df['Q1.6'].replace({3: 4, 2: 1})


array([4, 1])

### Calcolo di nuovo le performance

In [39]:
opzionale = False

results.loc[14] = [
    inter_annotator_agreement(df, 'Q1.6_binary', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df, 'Q1.6_binary', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.6_binary', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.6_binary', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.6_binary', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.6_binary', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]


results


,Agreement Manuel vs. Angelo,Agreement Manuel vs. Marinella,Agreement Marinella vs. Angelo,Agreement Manuel vs. ChatGPT,Agreement Marinella vs. chatGPT,Agreement Angelo vs. chatGPT,Interpretazione K
0,0.690347,0.730706,0.825852,0.490447,0.392159,0.394659,
1,0.452055,0.355599,0.428279,0.219982,0.233129,0.169449,
2,0.735294,0.778922,0.761538,0.045519,0.039092,0.044853,
3,0.266304,0.853081,0.655556,0.484663,0.409792,0.418538,
4,0.870370,0.916532,0.889993,0.293578,0.364916,0.274823,
5,0.859130,0.873469,0.920851,0.706326,0.845737,0.757073,
6,0.492958,0.663043,0.661818,0.238566,0.172932,0.143342,
7,NaN,0.000000,NaN,NaN,-0.004566,0.000000,
8,0.000000,NaN,0.000000,0.000000,0.393382,-0.004975,
9,0.390977,0.884758,0.728070,0.491749,0.550409,0.550667,


# Q 1.7

## Questions

### Original

Q1.7: Does this story negatively target a specific group or individual? (Check one):<br>
    1.  No<br>
    2.  Yes<br>


### Reviseted

Step 14 - Does this story negatively target a specific group or individual?:<br>
    1.  No<br>
    2.  Yes<br>


## Analisi

In [40]:
df['Q1.7'].unique()

array([ 1,  2, -1])

### K

In [41]:
opzionale = False

results.loc[15] = [
    inter_annotator_agreement(df, 'Q1.7', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df, 'Q1.7', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.7', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df, 'Q1.7', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.7', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df, 'Q1.7', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]


results


,Agreement Manuel vs. Angelo,Agreement Manuel vs. Marinella,Agreement Marinella vs. Angelo,Agreement Manuel vs. ChatGPT,Agreement Marinella vs. chatGPT,Agreement Angelo vs. chatGPT,Interpretazione K
0,0.690347,0.730706,0.825852,0.490447,0.392159,0.394659,
1,0.452055,0.355599,0.428279,0.219982,0.233129,0.169449,
2,0.735294,0.778922,0.761538,0.045519,0.039092,0.044853,
3,0.266304,0.853081,0.655556,0.484663,0.409792,0.418538,
4,0.870370,0.916532,0.889993,0.293578,0.364916,0.274823,
5,0.859130,0.873469,0.920851,0.706326,0.845737,0.757073,
6,0.492958,0.663043,0.661818,0.238566,0.172932,0.143342,
7,NaN,0.000000,NaN,NaN,-0.004566,0.000000,
8,0.000000,NaN,0.000000,0.000000,0.393382,-0.004975,
9,0.390977,0.884758,0.728070,0.491749,0.550409,0.550667,


### CM

In [42]:
columns_to_consider = ["Q1.7"]
possible_values = {
    'Q1.7': ['-1', '1', '2'],
}

#conf_matrices = calculate_confusion_matrix('Manuel','ChatGPT4AsAnnotator', columns_to_consider, possible_values)


# Q 1.7.1

## Questions

### Original

Q1.7.1: If the answer 1.7 is yes, please specify the group or individual negatively target. (Check all that apply):<br>
0.  Conservatives<br>
1.  Liberals<br>
2.  Gender<br>
3.  LGBTQ<br>
4.  Immigrants<br>
5.  Jews/Judaism<br>
6.  Islam/Muslims<br>
7.  Christians/Christianity<br>
8.  Religion (Other)<br>
9.  Race/ethnicity<br>
10.  Reputation (Organisation)<br>
11.  Reputation (Person)<br>
12.  Other<br>


### Reviseted

Step 15 - Specify the group or individual negatively targeted. Specify for each option "Yes" if the element appears in the title, "No" oyherwise):<br>
    0.  Political<br>
    1.  Religion<br>
    2.  Race/ethnicity<br>
    3.  Orientation<br>
    4.  The article negatively targets other groups or minorities<br>


## Analisi

In [43]:
df['Q1.7.1'].unique()

array(['-1', '3', '4', '0', '1', '1;3', '2', '1;4', nan, '3;2;4', '3;2',
       '6', '5', '1;3;5', '1;6', '4;5', '3;6', '3;5', '4;6', '2;5',
       '1;4;6', '2;6', '2;3;5;6', '2;3;5', '1;3;4;5'], dtype=object)

### K

In [44]:
opzionale = True

print(inter_annotator_agreement(df, 'Q1.7.1', 'Manuel', 'Angelo', opzionale))
print(inter_annotator_agreement(df, 'Q1.7.1', 'Manuel', 'Marinella', opzionale))
print(inter_annotator_agreement(df, 'Q1.7.1', 'Angelo', 'Marinella', opzionale))
print('----------------------------------')
print(inter_annotator_agreement(df, 'Q1.7.1', 'Manuel', 'ChatGPT4AsAnnotator', opzionale))
print(inter_annotator_agreement(df, 'Q1.7.1', 'Marinella', 'ChatGPT4AsAnnotator', opzionale))
print(inter_annotator_agreement(df, 'Q1.7.1', 'Angelo', 'ChatGPT4AsAnnotator', opzionale))


0.509090909090909
0.4777809227285593
0.4040485829959515
----------------------------------
-0.0003374388392105576
-0.0003157296512241725
0.0


### Creo colonne singole per ciascun valore possibile nella colonna e salvo in un nuovo dataframe

In [45]:
# Dividi la colonna 'Q1.3' in colonne separate
df_split = df['Q1.7.1'].str.get_dummies(sep=';')

# Rinomina le colonne con i valori unici
df_split.columns = ['Q1.7.1_' + col for col in df_split.columns]

# Unisci il dataframe diviso con il dataframe originale
df_1_7_1 = pd.concat([df, df_split], axis=1)

# Mostra il dataframe risultante
df_1_7_1 = df_1_7_1.loc[:, ['id','annotator','Q1.7.1_-1','Q1.7.1_0','Q1.7.1_1','Q1.7.1_2','Q1.7.1_3','Q1.7.1_4']]

df_1_7_1

,id,annotator,Q1.7.1_-1,Q1.7.1_0,Q1.7.1_1,Q1.7.1_2,Q1.7.1_3,Q1.7.1_4
0,18d71db7-f362-46fa-953e-d8353c7f7232,Manuel,1,0,0,0,0,0
1,18d71db7-f362-46fa-953e-d8353c7f7232,Angelo,1,0,0,0,0,0
2,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Manuel,1,0,0,0,0,0
3,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Marinella,1,0,0,0,0,0
4,34b92f4c-32db-454c-a369-ca2032443f6c,Manuel,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1014,3696bc79-2f24-47e2-9f13-fabd92b7a271,ChatGPT4AsAnnotator,0,0,0,0,0,0
1015,89ca8966-8ade-49fd-a036-c528a1f7871c,ChatGPT4AsAnnotator,0,0,0,0,0,0
1016,a425710b-96e3-4840-87be-75deea62775d,ChatGPT4AsAnnotator,0,0,0,0,0,0
1017,df86ebbe-1672-4c99-8895-b0fe742ed00c,ChatGPT4AsAnnotator,0,0,0,0,0,0


### Rifaccio il K

In [46]:
opzionale = True




results.loc[16] = [
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_0', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_0', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_0', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_0', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_0', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_0', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]
results.loc[17] = [
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_1', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_1', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_1', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_1', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_1', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_1', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]
results.loc[18] = [
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_2', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_2', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_2', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_2', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_2', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_2', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]
results.loc[19] = [
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_3', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_3', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_3', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_3', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_3', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_3', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]
results.loc[20] = [
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_4', 'Manuel', 'Angelo', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_4', 'Manuel', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_4', 'Angelo', 'Marinella', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_4', 'Manuel', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_4', 'Marinella', 'ChatGPT4AsAnnotator', opzionale),
    inter_annotator_agreement(df_1_7_1, 'Q1.7.1_4', 'Angelo', 'ChatGPT4AsAnnotator', opzionale),
    ""
]

results


/home/john/Progetti/ChatGPTAsAnnotator/ChatGPTAsAnnotator/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/john/Progetti/ChatGPTAsAnnotator/ChatGPTAsAnnotator/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:708: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/john/Progetti/ChatGPTAsAnnotator/ChatGPTAsAnnotator/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/john/Progetti/ChatGPTAsAnnotator/ChatGPTAsAnnotator/.venv/lib/python3.12/site-packages/sklearn/metrics/_c

,Agreement Manuel vs. Angelo,Agreement Manuel vs. Marinella,Agreement Marinella vs. Angelo,Agreement Manuel vs. ChatGPT,Agreement Marinella vs. chatGPT,Agreement Angelo vs. chatGPT,Interpretazione K
0,0.690347,0.730706,0.825852,0.490447,0.392159,0.394659,
1,0.452055,0.355599,0.428279,0.219982,0.233129,0.169449,
2,0.735294,0.778922,0.761538,0.045519,0.039092,0.044853,
3,0.266304,0.853081,0.655556,0.484663,0.409792,0.418538,
4,0.870370,0.916532,0.889993,0.293578,0.364916,0.274823,
5,0.859130,0.873469,0.920851,0.706326,0.845737,0.757073,
6,0.492958,0.663043,0.661818,0.238566,0.172932,0.143342,
7,NaN,0.000000,NaN,NaN,-0.004566,0.000000,
8,0.000000,NaN,0.000000,0.000000,0.393382,-0.004975,
9,0.390977,0.884758,0.728070,0.491749,0.550409,0.550667,


### Faccio la balanced accuracy

In [47]:
#calculate_standard_metrics('Manuel','ChatGPT4AsAnnotator', df_1_7_1)


In [48]:
#calculate_standard_metrics('Marinella','ChatGPT4AsAnnotator', df_1_7_1)


In [49]:
#calculate_standard_metrics('Angelo','ChatGPT4AsAnnotator', df_1_7_1)


In [50]:
# Funzione per determinare il livello di accordo
def get_agreement_level(value):
    if value > 0.90:
        return "Almost Perfect"
    elif value > 0.80:
        return "Strong"
    elif value > 0.60:
        return "Moderate"
    elif value > 0.40:
        return "Weak"
    elif value > 0.21:
        return "Minimal"
    else:
        return "None"

# Trova il valore massimo tra le colonne specificate e assegna il livello di accordo
columns_to_check = ["Agreement Manuel vs. ChatGPT", "Agreement Marinella vs. chatGPT", "Agreement Angelo vs. chatGPT"]
results["Interpretazione K"] = results[columns_to_check].max(axis=1).apply(get_agreement_level)

results

,Agreement Manuel vs. Angelo,Agreement Manuel vs. Marinella,Agreement Marinella vs. Angelo,Agreement Manuel vs. ChatGPT,Agreement Marinella vs. chatGPT,Agreement Angelo vs. chatGPT,Interpretazione K
0,0.690347,0.730706,0.825852,0.490447,0.392159,0.394659,Weak
1,0.452055,0.355599,0.428279,0.219982,0.233129,0.169449,Minimal
2,0.735294,0.778922,0.761538,0.045519,0.039092,0.044853,None
3,0.266304,0.853081,0.655556,0.484663,0.409792,0.418538,Weak
4,0.870370,0.916532,0.889993,0.293578,0.364916,0.274823,Minimal
5,0.859130,0.873469,0.920851,0.706326,0.845737,0.757073,Strong
6,0.492958,0.663043,0.661818,0.238566,0.172932,0.143342,Minimal
7,NaN,0.000000,NaN,NaN,-0.004566,0.000000,None
8,0.000000,NaN,0.000000,0.000000,0.393382,-0.004975,Minimal
9,0.390977,0.884758,0.728070,0.491749,0.550409,0.550667,Weak


In [51]:
from odf.opendocument import OpenDocumentSpreadsheet
from odf.style import Style, TableCellProperties, TableColumnProperties, TableRowProperties
from odf.table import Table, TableRow, TableCell, TableColumn
from odf.text import P

def get_color(interpretation):
    color_mapping = {
        'None': '#FF0000',     # Rosso
        'Minimal': '#FFA500',  # Arancione
        'Weak': '#FFFF00',     # Giallo
        'Moderate': '#ADFF2F', # Verde chiaro
        'Strong': '#008000'    # Verde
    }
    return color_mapping.get(interpretation, '#FFFFFF')  # Default to white if interpretation not found

# Creare il documento ODS
doc = OpenDocumentSpreadsheet()
table = Table(name="results")

# Aggiungere le intestazioni delle colonne
header_row = TableRow()
for column in results.columns:
    cell = TableCell()
    cell.addElement(P(text=column))
    header_row.addElement(cell)
table.addElement(header_row)

# Aggiungere i dati e colorare le celle nella colonna "Interpretazione K"
for row in results.itertuples(index=False):
    table_row = TableRow()
    for i, value in enumerate(row):
        cell = TableCell()
        cell.addElement(P(text=str(value)))
        if results.columns[i] == 'Interpretazione K':
            color = get_color(value)
            cell_style = Style(name=f"CellStyle_{value}", family="table-cell")
            cell_style.addElement(TableCellProperties(backgroundcolor=color))
            doc.styles.addElement(cell_style)
            cell.setAttribute("stylename", cell_style)
        table_row.addElement(cell)
    table.addElement(table_row)

# Aggiungere la tabella al documento
doc.spreadsheet.addElement(table)

# Salvare il documento
doc.save("Results.ods")
